In [12]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
import numpy as np
import ray
from ray.rllib import agents
from tqdm.notebook import tqdm
import random
from ray.rllib.policy.policy import Policy
from gym.spaces import Discrete, Box
from ray.rllib.agents.ppo import PPOTrainer
from functools import partial
from ray.tune.logger import pretty_print
from ray.rllib.agents.ppo.ppo_tf_policy import PPOTFPolicy
from ray.rllib.models import ModelCatalog


import ray
from ray import tune
from ray.tune import track

import math
import gym

from gym_compete_to_rllib import created_envs, env_name, create_env, env_name_rllib

In [13]:
env_cls = create_env
env_config = {'with_video': True}

def build_trainer_config(restore_state=None, train_policies=None, config=None):
    """Build configuration for 1 run."""
    obs_space = env_cls(env_config).observation_space
    act_space = env_cls(env_config).action_space

    policy_template = "player_%d"

    def get_agent_config(agent_id):
        agent_config = (PPOTFPolicy, obs_space, act_space, {
            'model': {
                        "custom_model": "GymCompetePretrainedModel",
                        "custom_model_config": {
                            "agent_id": agent_id - 1,
                            "env_name": env_name,
                            "model_config": {},
                            "name": "model_%s" % (agent_id - 1)
                        },           
                        
                    },
            
            "framework": "tfe",
        })
        
        return agent_config
        
#         agent_config = (PPOTFPolicy, obs_space, act_space, {
#                     "model": {
#                         "use_lstm": False,
#                         "fcnet_hiddens": [64, 64],
#                         #"custom_action_dist": "DiagGaussian",
#                     },
#                     "framework": "tfe",
#                 })
        
#         return agent_config

    N_POLICIES = 2

    policies = {policy_template % i: get_agent_config(i) for i in range(1, 1  + N_POLICIES)}
    policies_keys = list(sorted(policies.keys()))

    def select_policy(agent_id):
        assert agent_id in ["player_1", "player_2"]
        agent_ids = ["player_1", "player_2"]
        
        # selecting the corresponding policy (only for 2 policies)
        return policies_keys[agent_ids.index(agent_id)]

        # randomly choosing an opponent
        # return np.random.choice(list(policies.keys()))
    
    if train_policies is None:
        train_policies = list(policies.keys())
        
    for k in train_policies:
        assert k in policies.keys()

    config = {
        "env": env_name_rllib,
        "env_config": env_config,
    #    "gamma": 0.9,
      "num_workers": 0,
    #  "num_envs_per_worker": 10,
    #   "rollout_fragment_length": 10,
       "train_batch_size": config['train_batch_size'],
        "multiagent": {
            "policies_to_train": train_policies,
            "policies": policies,
            "policy_mapping_fn": select_policy,
        },
        "framework": "tfe",
        #"train_batch_size": 512
        #"num_cpus_per_worker": 2
    }
    return config


def build_trainer(restore_state=None, train_policies=None, config=None):
    """Create a RPS trainer for 2 agents, restore state, and only train specific policies."""
    
    print("Using config")
    print(config)
    cls = PPOTrainer
    trainer = cls(config=config)
    env = trainer.workers.local_worker().env
    if restore_state is not None:
        trainer.restore_from_object(restore_state)
    return trainer

def train(trainer, stop_iters, do_track=True):
    """Train the agents and return the state of the trainer."""
    for _ in range(stop_iters):
        results = trainer.train()
        print(pretty_print(results))
        if do_track:
            track.log(**results)
    o = trainer.save_to_object()
    return o

trainer = None

def train_one(config, restore_state=None, do_track=True):
    print(config)
    rl_config = build_trainer_config(restore_state=restore_state,
                              train_policies=config['train_policies'],
                              config=config)
    global trainer
    trainer = build_trainer(restore_state=None, config=rl_config)
    train(trainer, config['train_steps'], do_track=do_track)


# try changing learning rate
config = {'train_batch_size': 128}

config['train_steps'] = 10
config['train_policies'] = [] #['player_1', 'player_2']
config['num_workers'] = 3

In [14]:
train_one(config, do_track=False)

{'train_batch_size': 128, 'train_steps': 10, 'train_policies': [], 'num_workers': 3}
Creating agent humanoid_blocker
Reading agent XML from: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Creating agent humanoid
Reading agent XML from: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Scene XML path: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/world_body.humanoid_body.humanoid_body.xml
Created Scene with agents
Creating agent humanoid_blocker
Reading agent XML from: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Creating agent humanoid
Reading agent XML from: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Scene XML path: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/world_body.humanoid_body.humanoid_body.xml
Created Scene with agents


pip install 'ray[tune]' to see TensorBoard files.
Could not instantiate TBXLogger: No module named 'tensorboardX'.


Using config
{'env': 'YouShallNotPassHumans-v0_rllib', 'env_config': {'with_video': True}, 'num_workers': 0, 'train_batch_size': 128, 'multiagent': {'policies_to_train': [], 'policies': {'player_1': (<class 'ray.rllib.policy.tf_policy_template.PPOTFPolicy'>, Box(380,), Box(17,), {'model': {'custom_model': 'GymCompetePretrainedModel', 'custom_model_config': {'agent_id': 0, 'env_name': 'multicomp/YouShallNotPassHumans-v0', 'model_config': {}, 'name': 'model_0'}}, 'framework': 'tfe'}), 'player_2': (<class 'ray.rllib.policy.tf_policy_template.PPOTFPolicy'>, Box(380,), Box(17,), {'model': {'custom_model': 'GymCompetePretrainedModel', 'custom_model_config': {'agent_id': 1, 'env_name': 'multicomp/YouShallNotPassHumans-v0', 'model_config': {}, 'name': 'model_1'}}, 'framework': 'tfe'})}, 'policy_mapping_fn': <function build_trainer_config.<locals>.select_policy at 0x7f8de9831e50>}, 'framework': 'tfe'}
Creating agent humanoid_blocker
Reading agent XML from: /home/sergei/git/chai/multiagent-compe

Install gputil for GPU system monitoring.



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for

In [15]:
from load_gym_compete_policy import get_policy_value_nets

In [17]:
obs = env_cls(env_config).reset()

Creating agent humanoid_blocker
Reading agent XML from: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Creating agent humanoid
Reading agent XML from: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Scene XML path: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/world_body.humanoid_body.humanoid_body.xml
Created Scene with agents


In [18]:
acts = [trainer.compute_action(obs['player_1'], policy_id='player_1') for _ in range(1000)]

In [19]:
np.mean(acts, axis=0)

array([-0.3999962 ,  0.3999962 , -0.39984488, -0.3999962 ,  0.39890635,
       -0.36216214,  0.01549192, -0.39968854,  0.22517362,  0.18174249,
       -0.00849053,  0.37303784, -0.39654046, -0.39927378, -0.366909  ,
        0.39690655, -0.3999962 ], dtype=float32)

In [20]:
nets = get_policy_value_nets(env_name, 0)
policy_net_orig = nets['policy_mean_logstd_flat']

Creating agent humanoid_blocker
Reading agent XML from: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Creating agent humanoid
Reading agent XML from: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Scene XML path: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/world_body.humanoid_body.humanoid_body.xml
Created Scene with agents
tf eager True
obsmean <class 'numpy.ndarray'> (380,)
tf eager True
Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 380)]        0                                            
__________________________________________________________________________________________________
observation_preprocessing_layer (None, 380)          761         input_3[0][

In [21]:
m = trainer.get_policy('player_1').model
m.policy_net(obs['player_1'].reshape(1, -1))

<tf.Tensor: shape=(1, 34), dtype=float32, numpy=
array([[-2.644432  ,  1.6587822 , -1.3111818 , -2.5256457 ,  1.0890175 ,
        -0.632396  ,  0.02013881, -1.5281134 ,  0.31184098,  0.24941634,
        -0.01156497,  0.69752496, -1.1240312 , -1.470026  , -0.8531909 ,
         0.9602913 , -2.8689044 , -1.200372  , -1.0477431 , -1.2787865 ,
        -1.0475501 , -1.1048975 , -1.2324022 , -1.5336553 , -0.9083957 ,
        -1.0634612 , -1.1298783 , -1.6375971 , -1.1652852 , -0.99094224,
        -0.85480416, -0.82656866, -1.1858863 , -0.9486331 ]],
      dtype=float32)>

In [22]:
nets['policy_mean_logstd'](obs['player_1'].reshape(1, -1))

<tf.Tensor: shape=(1, 17, 2), dtype=float32, numpy=
array([[[-2.6844046 , -1.1979917 ],
        [ 1.6134702 , -1.0522642 ],
        [-1.2499714 , -1.2768428 ],
        [-2.5103867 , -1.0497713 ],
        [ 1.2004242 , -1.1014946 ],
        [-0.6241886 , -1.2318319 ],
        [ 0.01984857, -1.5337685 ],
        [-1.5142651 , -0.9082957 ],
        [ 0.31476304, -1.0616486 ],
        [ 0.1913874 , -1.1258026 ],
        [-0.02376649, -1.6367302 ],
        [ 0.7484977 , -1.1637001 ],
        [-1.1188809 , -0.9924653 ],
        [-1.473376  , -0.85632914],
        [-0.8569962 , -0.82374895],
        [ 0.9788799 , -1.1841927 ],
        [-2.9387057 , -0.9448246 ]]], dtype=float32)>

In [23]:
mean, logstd = tf.split(policy_net_orig(obs['player_1'].reshape(1, -1)), 2, axis=1)

In [24]:
np.mean([np.clip(np.random.normal(loc=mean[0], scale=np.exp(logstd[0]), size=(17,)), -0.4, 0.4) for _ in range(1000)], axis=0)

array([-0.4       ,  0.4       , -0.39953831, -0.4       ,  0.39932675,
       -0.3645706 ,  0.0283459 , -0.39952335,  0.21959492,  0.14225847,
       -0.03031551,  0.37967087, -0.39714089, -0.39899308, -0.36727401,
        0.39759381, -0.4       ])

In [25]:
np.mean([trainer.compute_action(obs['player_1'], policy_id='player_1') for _ in range(1000)], axis=0)

array([-0.3999962 ,  0.3999962 , -0.3999962 , -0.3999962 ,  0.39809477,
       -0.363111  ,  0.00771117, -0.39989665,  0.20237626,  0.18369985,
       -0.01843473,  0.3739839 , -0.39581993, -0.39893857, -0.36984372,
        0.39702076, -0.3999962 ], dtype=float32)

In [26]:
trainer.compute_action(obs['player_1'], policy_id='player_1')

array([-4.0000001e-01,  4.0000001e-01, -4.0000001e-01, -4.0000001e-01,
        4.0000001e-01, -4.0000001e-01,  2.8095581e-04, -4.0000001e-01,
        1.8436955e-01, -7.9319030e-03,  5.6343660e-02,  4.0000001e-01,
       -4.0000001e-01, -4.0000001e-01, -4.0000001e-01,  4.0000001e-01,
       -4.0000001e-01], dtype=float32)

In [28]:
env_cls(env_config).action_space.high

Creating agent humanoid_blocker
Reading agent XML from: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Creating agent humanoid
Reading agent XML from: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Scene XML path: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/world_body.humanoid_body.humanoid_body.xml
Created Scene with agents


array([0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4,
       0.4, 0.4, 0.4, 0.4], dtype=float32)